# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [1]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np

import nsfg
import first

ModuleNotFoundError: No module named 'thinkstats2'

Given a list of values, there are several ways to count the frequency of each value.

In [ ]:
t = [1, 2, 2, 3, 5]

You can use a Python dictionary:

In [ ]:
hist = {}
for x in t:
    hist[x] = hist.get(x, 0) + 1
    
hist

You can use a `Counter` (which is a dictionary with additional methods):

In [ ]:
from collections import Counter
counter = Counter(t)
counter

Or you can use the `Hist` object provided by `thinkstats2`:

In [ ]:
import thinkstats2
hist = thinkstats2.Hist([1, 2, 2, 3, 5])
hist

`Hist` provides `Freq`, which looks up the frequency of a value.

In [ ]:
hist.Freq(2)

You can also use the bracket operator, which does the same thing.

In [ ]:
hist[2]

If the value does not appear, it has frequency 0.

In [ ]:
hist[4]

The `Values` method returns the values:

In [ ]:
hist.Values()

So you can iterate the values and their frequencies like this:

In [ ]:
for val in sorted(hist.Values()):
    print(val, hist[val])

Or you can use the `Items` method:

In [ ]:
for val, freq in hist.Items():
     print(val, freq)

`thinkplot` is a wrapper for `matplotlib` that provides functions that work with the objects in `thinkstats2`.

For example `Hist` plots the values and their frequencies as a bar graph.

`Config` takes parameters that label the x and y axes, among other things.

In [ ]:
import thinkplot
thinkplot.Hist(hist)
thinkplot.Config(xlabel='value', ylabel='frequency')

As an example, I'll replicate some of the figures from the book.

First, I'll load the data from the pregnancy file and select the records for live births.

In [ ]:
preg = nsfg.ReadFemPreg()
live = preg[preg.outcome == 1]

Here's the histogram of birth weights in pounds.  Notice that `Hist` works with anything iterable, including a Pandas Series.  The `label` attribute appears in the legend when you plot the `Hist`. 

In [ ]:
hist = thinkstats2.Hist(live.birthwgt_lb, label='birthwgt_lb')
thinkplot.Hist(hist)
thinkplot.Config(xlabel='Birth weight (pounds)', ylabel='Count')

Before plotting the ages, I'll apply `floor` to round down:

In [ ]:
ages = np.floor(live.agepreg)

In [ ]:
hist = thinkstats2.Hist(ages, label='agepreg')
thinkplot.Hist(hist)
thinkplot.Config(xlabel='years', ylabel='Count')

As an exercise, plot the histogram of pregnancy lengths (column `prglngth`).

In [ ]:
# Solution

hist = thinkstats2.Hist(live.prglngth, label='prglngth')
thinkplot.Hist(hist)
thinkplot.Config(xlabel='weeks', ylabel='Count')

`Hist` provides smallest, which select the lowest values and their frequencies.

In [ ]:
for weeks, freq in hist.Smallest(10):
    print(weeks, freq)

Use `Largest` to display the longest pregnancy lengths.

In [ ]:
# Solution

for weeks, freq in hist.Largest(10):
    print(weeks, freq)

From live births, we can selection first babies and others using `birthord`, then compute histograms of pregnancy length for the two groups.

In [ ]:
firsts = live[live.birthord == 1]
others = live[live.birthord != 1]

first_hist = thinkstats2.Hist(firsts.prglngth, label='first')
other_hist = thinkstats2.Hist(others.prglngth, label='other')

We can use `width` and `align` to plot two histograms side-by-side.

In [ ]:
width = 0.45
thinkplot.PrePlot(2)
thinkplot.Hist(first_hist, align='right', width=width)
thinkplot.Hist(other_hist, align='left', width=width)
thinkplot.Config(xlabel='weeks', ylabel='Count', xlim=[27, 46])

`Series` provides methods to compute summary statistics:

In [ ]:
mean = live.prglngth.mean()
var = live.prglngth.var()
std = live.prglngth.std()

Here are the mean and standard deviation:

In [ ]:
mean, std

As an exercise, confirm that `std` is the square root of `var`:

In [ ]:
# Solution

np.sqrt(var) == std

Here's are the mean pregnancy lengths for first babies and others:

In [ ]:
firsts.prglngth.mean(), others.prglngth.mean()

And here's the difference (in weeks):

In [ ]:
firsts.prglngth.mean() - others.prglngth.mean()

This functon computes the Cohen effect size, which is the difference in means expressed in number of standard deviations:

In [ ]:
def CohenEffectSize(group1, group2):
    """Computes Cohen's effect size for two groups.
    
    group1: Series or DataFrame
    group2: Series or DataFrame
    
    returns: float if the arguments are Series;
             Series if the arguments are DataFrames
    """
    diff = group1.mean() - group2.mean()

    var1 = group1.var()
    var2 = group2.var()
    n1, n2 = len(group1), len(group2)

    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    d = diff / np.sqrt(pooled_var)
    return d

Compute the Cohen effect size for the difference in pregnancy length for first babies and others.

In [ ]:
# Solution

CohenEffectSize(firsts.prglngth, others.prglngth)

## Exercises

Using the variable `totalwgt_lb`, investigate whether first babies are lighter or heavier than others. 

Compute Cohen’s effect size to quantify the difference between the groups.  How does it compare to the difference in pregnancy length?

In [ ]:
# Solution

firsts.totalwgt_lb.mean(), others.totalwgt_lb.mean()

In [ ]:
# Solution

CohenEffectSize(firsts.totalwgt_lb, others.totalwgt_lb)

For the next few exercises, we'll load the respondent file:

In [ ]:
resp = nsfg.ReadFemResp()

Make a histogram of <tt>totincr</tt> the total income for the respondent's family.  To interpret the codes see the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=FEM&section=R&subSec=7876&srtLabel=607543).

In [ ]:
# Solution

hist = thinkstats2.Hist(resp.totincr)
thinkplot.Hist(hist, label='totincr')
thinkplot.Config(xlabel='income (category)', ylabel='Count')

Make a histogram of <tt>age_r</tt>, the respondent's age at the time of interview.

In [ ]:
# Solution

hist = thinkstats2.Hist(resp.ager)
thinkplot.Hist(hist, label='ager')
thinkplot.Config(xlabel='age (years)', ylabel='Count')

Make a histogram of <tt>numfmhh</tt>, the number of people in the respondent's household.

In [ ]:
# Solution

hist = thinkstats2.Hist(resp.numfmhh)
thinkplot.Hist(hist, label='numfmhh')
thinkplot.Config(xlabel='number of people', ylabel='Count')

Make a histogram of <tt>parity</tt>, the number of children borne by the respondent.  How would you describe this distribution?

In [ ]:
# Solution

# This distribution is positive-valued and skewed to the right.

hist = thinkstats2.Hist(resp.parity)
thinkplot.Hist(hist, label='parity')
thinkplot.Config(xlabel='parity', ylabel='Count')

Use Hist.Largest to find the largest values of <tt>parity</tt>.

In [ ]:
# Solution

hist.Largest(10)

Let's investigate whether people with higher income have higher parity.  Keep in mind that in this study, we are observing different people at different times during their lives, so this data is not the best choice for answering this question.  But for now let's take it at face value.

Use <tt>totincr</tt> to select the respondents with the highest income (level 14).  Plot the histogram of <tt>parity</tt> for just the high income respondents.

In [ ]:
# Solution

rich = resp[resp.totincr == 14]
hist = thinkstats2.Hist(rich.parity)
thinkplot.Hist(hist, label='parity')
thinkplot.Config(xlabel='parity', ylabel='Count')

Find the largest parities for high income respondents.

In [ ]:
# Solution

hist.Largest(10)

Compare the mean <tt>parity</tt> for high income respondents and others.

In [ ]:
# Solution

not_rich = resp[resp.totincr < 14]
rich.parity.mean(), not_rich.parity.mean()

Compute the Cohen effect size for this difference.  How does it compare with the difference in pregnancy length for first babies and others?

In [ ]:
# Solution

# This effect is about 10 times stronger than the difference in pregnancy length.
# But remembering the design of the study, we should not make too much of this
# apparent effect.

CohenEffectSize(rich.parity, not_rich.parity)